In [1]:
import requests
import xml.etree.ElementTree as ET
import pandas as pd
#지진해일 대피장소 data
# XML 데이터를 가져오기 위한 요청 설정
service_key = 'mwomjlIWqcy1kXY9c3tcT8s7lobXokh4tnki7vMVgD4qjX+k2yfZR+nLzbEsGgfDhxg6ShX4h7/6eCqJpajr5g=='
url = 'http://apis.data.go.kr/1741000/TsunamiShelter3/getTsunamiShelter1List'
params = {'serviceKey': service_key, 'pageNo': '', 'numOfRows': '654', 'type': 'xml'}
response = requests.get(url, params=params)
xml_data = response.content

# XML 데이터 파싱
root = ET.fromstring(xml_data)
rows = root.findall('.//row')

# 추출한 데이터를 저장할 리스트
data = []

# XML 데이터에서 필요한 필드 추출
for row in rows:
    record = {}
    for element in row.iter():
        record[element.tag] = element.text
    data.append(record)

# 데이터프레임 생성
df = pd.DataFrame(data)
del df['row']
del df['id']
del df['remarks']
df=df.drop(['lenth','shel_div_type'],axis=1)
df=df.sort_values(by=['sido_name','sigungu_name'])
df.to_csv('csv_file/지진해일대피소.csv', index=False, encoding='utf-8')
df

,sido_name,sigungu_name,shel_nm,address,lon,lat,shel_av,seismic,height
67,강원도,강릉시,씨마크호텔,강원도 강릉시 강문동 274-1,128.91539400,37.79799200,700,1등급,26
68,강원도,강릉시,세인트컨벤션웨딩홀,강원도 강릉시 성덕동 42,128.94970900,37.76512600,1870,2등급,12
72,강원도,강릉시,현대아파트 앞 야산,강원도 강릉시 경포동 산10,128.89958800,37.80132800,2000,None,10
74,강원도,강릉시,송정해변 신도브래뉴아파트,강원도 강릉시 송정동 289,128.93791300,37.77579600,3500,1등급,6
75,강원도,강릉시,죽도봉,강원도 강릉시 송정동 11-1,128.95155400,37.76976900,800,None,33
...,...,...,...,...,...,...,...,...,...
642,전라남도,화순군,아산초등학교 운동장,전라남도 화순군 백아면 이천리 55번지,127.13334775,35.16877658,3030,None,0
643,전라남도,화순군,춘양초등학교 운동장,전라남도 화순군 춘양면 춘양로 124,126.95968839,34.94956153,5576,None,0
645,전라남도,화순군,도곡초등학교 운동장,전라남도 화순군 도곡면 천암1길 71,126.90860000,35.01310000,5091,None,0
647,전라남도,화순군,도암중학교 운동장,전라남도 화순군 도암면 권동길 4-6,126.89550000,34.93780000,4848,None,0


In [2]:
import requests
import xml.etree.ElementTree as ET
import pandas as pd
# 옥외 대피장소 data
# XML 데이터를 가져오기 위한 요청 설정
service_key = 'mwomjlIWqcy1kXY9c3tcT8s7lobXokh4tnki7vMVgD4qjX+k2yfZR+nLzbEsGgfDhxg6ShX4h7/6eCqJpajr5g=='
url = 'http://apis.data.go.kr/1741000/EmergencyAssemblyArea_Earthquake2/getArea1List'
params = {'serviceKey': service_key, 'pageNo': '1', 'numOfRows': '1000', 'type': 'xml'}
response = requests.get(url, params=params)
xml_data = response.content

# XML 데이터 파싱
root = ET.fromstring(xml_data)
total_count = int(root.find('.//totalCount').text)  # 전체 데이터 수
page_count = (total_count // 1000) + 1  # 페이지 수 계산

# 추출한 데이터를 저장할 리스트
data = []

# 페이지를 순차적으로 변경하면서 데이터 가져오기
for page in range(1, page_count + 1):
    params['pageNo'] = str(page)  # 페이지 번호 설정
    response = requests.get(url, params=params)
    xml_data = response.content

    # XML 데이터 파싱
    root = ET.fromstring(xml_data)
    rows = root.findall('.//row')

    # XML 데이터에서 필요한 필드 추출
    for row in rows:
        record = {}
        for element in row.iter():
            record[element.tag] = element.text
        data.append(record)
print(data)
# 데이터프레임 생성
df1 = pd.DataFrame(data)
del df1['row']
del df1['arcd']
del df1['acmdfclty_sn']
df1=df1.drop(['bdong_cd','hdong_cd'],axis=1)
df1 = df1.sort_values(by=['ctprvn_nm', 'sgg_nm'])
df1['rdnmadr_cd'] = df1['rdnmadr_cd'].str[:5]
df1['fclty_ar']=df1['fclty_ar'].astype(float)
df1['shel_av']=df1['fclty_ar']//3.3
df1['fclty_ar']=df1['fclty_ar'].astype(str)
df1 = df1.rename(columns={'ctprvn_nm':'sido_name','sgg_nm':'sigungu_name','vt_acmdfclty_nm':'shel_nm','xcord':'lon','ycord':'lat','dtl_adres':'address'})
df1.to_csv('csv_file/옥외대피장소.csv', index=False, encoding='utf-8')
df1

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



,sido_name,sigungu_name,shel_nm,rdnmadr_cd,address,fclty_ar,lon,lat,shel_av
7131,강원특별자치도,강릉시,구정초등학교,42150,강원특별자치도 강릉시 구정면 학산리 393-0,12065.0,128.891387595211,37.7175835666904,3656.0
7132,강원특별자치도,강릉시,강동초등학교,42150,강원특별자치도 강릉시 강동면 모전리 384-0,13604.0,127.480805980668,37.2944046773841,4122.0
7404,강원특별자치도,강릉시,왕산초등학교,42150,강원특별자치도 강릉시 왕산면 도마리 595-0,7179.0,128.836145008418,37.6734165886603,2175.0
7476,강원특별자치도,강릉시,옥계초등학교,42150,강원특별자치도 강릉시 옥계면 현내리 344-1,10000.0,129.03510240250688,37.61050834984292,3030.0
7477,강원특별자치도,강릉시,사천초등학교,42150,강원특별자치도 강릉시 사천면 미노리 406-0,8052.0,128.850647022792,37.8219235118247,2440.0
...,...,...,...,...,...,...,...,...,...
6220,충청북도,충주시,세성초등학교,43130,충청북도 충주시 살미면 세성리 147-0,4000.0,127.967988,36.904772,1212.0
6221,충청북도,충주시,수안보 생활체육공원,None,충청북도 충주시 수안보면 안보리 425-4,38400.0,127.968385,36.904360,11636.0
6222,충청북도,충주시,수안보초등학교 운동장,43130,충청북도 충주시 수안보면 안보리 598-0,12200.0,127.999423,36.844342,3696.0
6223,충청북도,충주시,금제공원,43130,충청북도 충주시 금릉동 691-0,7300.0,127.925520,36.989497,2212.0


In [3]:
# DataFrame을 추가
# 지역 코드 할당 안된 지역 할당 & 두 대피소 data 합치기
import pandas as pd
import re
df2 = pd.concat([df1, df], ignore_index=False)
df2 = df2.sort_values(by=['sido_name','sigungu_name'])
df2 = df2.drop_duplicates(subset=['lon', 'lat','shel_nm'])
df2 = df2.rename(columns={'rdnmadr_cd':'SIG_CD'})
df2 = df2[df2['shel_nm'].astype(bool)]
df2 = df2.sort_values(by=['sido_name', 'sigungu_name','SIG_CD'])
df2.loc[df2['sigungu_name'] == '연천군', 'SIG_CD'] = 41800
df2.loc[df2['sigungu_name'] == '창녕군', 'SIG_CD'] = 48740
df2.loc[df2['sigungu_name'] == '미추홀구', 'SIG_CD'] = 28177
df2.loc[df2['sigungu_name'] == '청주시 서원구', 'SIG_CD'] = 43112
df2.loc[df2['sigungu_name'] == '청주시 상당구', 'SIG_CD'] = 43111
df2.loc[df2['sigungu_name'] == '청주시 흥덕구', 'SIG_CD'] = 43113
df2.loc[df2['sigungu_name'] == '청주시 청원구', 'SIG_CD'] = 43114
df2['SIG_CD'].fillna(method='ffill', inplace=True)

df2.to_csv('csv_file/대피소 위치 data.csv', index=False, encoding='utf-8')
df2


,sido_name,sigungu_name,shel_nm,SIG_CD,address,fclty_ar,lon,lat,shel_av,seismic,height
50,강원도,강릉시,고성산,NaN,강원도 강릉시 강동면 정동진리 267-4번지,NaN,129.03378000,37.68960700,120,None,53
51,강원도,강릉시,정동초등학교,NaN,강원도 강릉시 강동면 정동진리 184,NaN,129.03357400,37.68722100,160,1등급,10
52,강원도,강릉시,메밀골동해막국수(정동진점),NaN,강원도 강릉시 강동면 정동진리 51-4,NaN,129.04094000,37.68101300,260,2등급,63
87,강원도,강릉시,기마봉 야산,NaN,강원도 강릉시 옥계면 금진리 96,NaN,129.04846800,37.65544100,2000,None,10
88,강원도,강릉시,옥계역,NaN,강원도 강릉시 옥계면 주수리 100,NaN,129.04971400,37.61689600,70,1등급,11
...,...,...,...,...,...,...,...,...,...,...,...
1677,충청북도,충주시,용산초등학교 운동장,43130,충청북도 충주시 용산동 1671,2500.0,127.944096,36.962318,757.0,NaN,NaN
1731,충청북도,충주시,앙성초등학교 운동장,43130,충청북도 충주시 앙성면 용대리 275,8000.0,127.757354,37.106116,2424.0,NaN,NaN
1732,충청북도,충주시,중원체육공원,43130,충청북도 충주시 중앙탑면 206-3,8050.0,127.856939,37.029979,2439.0,NaN,NaN
1734,충청북도,충주시,달천초등학교 운동장,43130,충청북도 충주시 용두동 411-1,6900.0,127.878912,36.962977,2090.0,NaN,NaN


In [4]:
import pandas as pd
import json
import geopandas as gpd
from fiona.crs import from_epsg
sigungu_geojson = 'json_file/SIGUNGU_2302.json'

with open(sigungu_geojson,'r',encoding="utf-8") as f:
    json_data = json.load(f)
    
gdf_pt_geom = gpd.points_from_xy(df2.lon, df2.lat)
shlter_gpd = gpd.GeoDataFrame(df2, geometry=gdf_pt_geom, crs=from_epsg(4326)) 


sigungu_gdf = gpd.read_file(sigungu_geojson)
sigungu_gdf = gpd.GeoDataFrame(sigungu_gdf, geometry=sigungu_gdf['geometry'], crs=from_epsg(4326))          # 시군구 geodataframe

shlter_data = gpd.sjoin(shlter_gpd, sigungu_gdf, how = "inner")

In [7]:
shlter_data

,sido_name,sigungu_name,shel_nm,SIG_CD_left,address,fclty_ar,lon,lat,shel_av,seismic,height,geometry,index_right,SIG_CD_right,SIG_ENG_NM,SIG_KOR_NM,CTP_KOR_NM
50,강원도,강릉시,고성산,NaN,강원도 강릉시 강동면 정동진리 267-4번지,NaN,129.03378000,37.68960700,120,None,53,POINT (129.03378 37.68961),119,42150,Gangneung-si,강릉시,강원도
51,강원도,강릉시,정동초등학교,NaN,강원도 강릉시 강동면 정동진리 184,NaN,129.03357400,37.68722100,160,1등급,10,POINT (129.03357 37.68722),119,42150,Gangneung-si,강릉시,강원도
52,강원도,강릉시,메밀골동해막국수(정동진점),NaN,강원도 강릉시 강동면 정동진리 51-4,NaN,129.04094000,37.68101300,260,2등급,63,POINT (129.04094 37.68101),119,42150,Gangneung-si,강릉시,강원도
87,강원도,강릉시,기마봉 야산,NaN,강원도 강릉시 옥계면 금진리 96,NaN,129.04846800,37.65544100,2000,None,10,POINT (129.04847 37.65544),119,42150,Gangneung-si,강릉시,강원도
88,강원도,강릉시,옥계역,NaN,강원도 강릉시 옥계면 주수리 100,NaN,129.04971400,37.61689600,70,1등급,11,POINT (129.04971 37.61690),119,42150,Gangneung-si,강릉시,강원도
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1677,충청북도,충주시,용산초등학교 운동장,43130,충청북도 충주시 용산동 1671,2500.0,127.944096,36.962318,757.0,NaN,NaN,POINT (127.94410 36.96232),139,43130,Chungju-si,충주시,충청북도
1731,충청북도,충주시,앙성초등학교 운동장,43130,충청북도 충주시 앙성면 용대리 275,8000.0,127.757354,37.106116,2424.0,NaN,NaN,POINT (127.75735 37.10612),139,43130,Chungju-si,충주시,충청북도
1732,충청북도,충주시,중원체육공원,43130,충청북도 충주시 중앙탑면 206-3,8050.0,127.856939,37.029979,2439.0,NaN,NaN,POINT (127.85694 37.02998),139,43130,Chungju-si,충주시,충청북도
1734,충청북도,충주시,달천초등학교 운동장,43130,충청북도 충주시 용두동 411-1,6900.0,127.878912,36.962977,2090.0,NaN,NaN,POINT (127.87891 36.96298),139,43130,Chungju-si,충주시,충청북도


In [20]:
for i in range(len(sigungu_gdf)): #시군구별 평균 구하기
    shel_av = shlter_data[sigungu_gdf['SIG_CD'][i] == shlter_data['SIG_CD_right']]['shel_av'].astype(int).sum()

    sigungu_gdf.loc[i,"shel_av"] = shel_av

In [21]:
sigungu_gdf

,SIG_CD,SIG_ENG_NM,SIG_KOR_NM,CTP_KOR_NM,geometry,shel_av
0,11110,Jongno-gu,종로구,서울특별시,"POLYGON ((127.01179 37.58157, 127.00864 37.580...",182700.0
1,11140,Jung-gu,중구,서울특별시,"POLYGON ((127.02337 37.57191, 127.01708 37.569...",30348.0
2,11170,Yongsan-gu,용산구,서울특별시,"POLYGON ((127.00899 37.54413, 127.00502 37.546...",69752.0
3,11200,Seongdong-gu,성동구,서울특별시,"POLYGON ((127.07238 37.55996, 127.07086 37.560...",84779.0
4,11215,Gwangjin-gu,광진구,서울특별시,"POLYGON ((127.11525 37.55676, 127.11230 37.559...",33010.0
...,...,...,...,...,...,...
245,48870,Hamyang-gun,함양군,경상남도,"POLYGON ((127.85639 35.56941, 127.85493 35.573...",54571.0
246,48880,Geochang-gun,거창군,경상남도,"POLYGON ((128.07736 35.83966, 128.07421 35.838...",31894.0
247,48890,Hapcheon-gun,합천군,경상남도,"POLYGON ((128.16371 35.77719, 128.16044 35.778...",162073.0
248,50110,Jeju-si,제주시,제주특별자치도,"MULTIPOLYGON (((126.63685 33.92129, 126.63761 ...",268359.0


In [22]:
sigungu_gdf["shel_av"].describe()

count       250.000000
mean     105151.148000
std       98143.638313
min        8091.000000
25%       45912.500000
50%       76459.500000
75%      129652.500000
max      658156.000000
Name: shel_av, dtype: float64

In [27]:
import pickle

with open('merge_gdf/sigungu_gdf_shelav.pickle', 'wb') as f:
        pickle.dump(sigungu_gdf, f)